# 성적 트렌드 분석

학생의 성적 변화 추이를 분석하고 취약 과목을 파악합니다.

## 분석 항목
1. 전체 성적 추이
2. 과목별 성적 분포
3. 등급별 분포
4. 취약 과목 분석
5. 학습-성적 상관관계

In [ ]:
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# 한글 폰트 설정
plt.rcParams['font.family'] = 'AppleGothic'  # macOS
plt.rcParams['axes.unicode_minus'] = False
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

In [ ]:
from db_connector import get_connector
from analysis import analyze_score_trends, calculate_study_efficiency

db = get_connector()
print('Supabase 연결 성공!')

## 1. 데이터 로드

In [ ]:
# 분석할 학생 ID
STUDENT_ID = 'YOUR_STUDENT_ID_HERE'

# 성적 데이터 조회
scores_df = db.get_student_scores(STUDENT_ID, limit=100)
print(f'성적 데이터 수: {len(scores_df)}')

if not scores_df.empty:
    scores_df['created_at'] = pd.to_datetime(scores_df['created_at'])
    scores_df = scores_df.sort_values('created_at')
    display(scores_df.head())

## 2. 전체 성적 추이

In [ ]:
if not scores_df.empty and 'score' in scores_df.columns:
    fig, ax = plt.subplots(figsize=(14, 6))
    
    # 전체 성적 추이
    ax.plot(scores_df['created_at'], scores_df['score'], 'o-', label='원점수', alpha=0.7)
    
    # 이동평균
    if len(scores_df) >= 5:
        scores_df['score_ma'] = scores_df['score'].rolling(window=5).mean()
        ax.plot(scores_df['created_at'], scores_df['score_ma'], '--', 
                label='5회 이동평균', linewidth=2, color='red')
    
    ax.set_xlabel('날짜')
    ax.set_ylabel('점수')
    ax.set_title('전체 성적 추이')
    ax.legend()
    ax.set_ylim(0, 100)
    
    # 트렌드 표시
    if len(scores_df) >= 2:
        z = np.polyfit(range(len(scores_df)), scores_df['score'], 1)
        trend = '상승' if z[0] > 0 else '하락'
        ax.text(0.02, 0.98, f'추세: {trend} ({z[0]:+.2f}점/회)', 
                transform=ax.transAxes, fontsize=12, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print('성적 데이터가 없습니다.')

## 3. 과목별 성적 분포

In [ ]:
if not scores_df.empty and 'subject' in scores_df.columns and 'score' in scores_df.columns:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # 과목별 박스플롯
    subject_order = scores_df.groupby('subject')['score'].mean().sort_values(ascending=False).index
    sns.boxplot(data=scores_df, x='subject', y='score', order=subject_order, ax=ax1, palette='Set2')
    ax1.set_xlabel('과목')
    ax1.set_ylabel('점수')
    ax1.set_title('과목별 성적 분포')
    ax1.tick_params(axis='x', rotation=45)
    ax1.axhline(y=scores_df['score'].mean(), color='red', linestyle='--', label='전체 평균')
    ax1.legend()
    
    # 과목별 평균 막대그래프
    subject_means = scores_df.groupby('subject')['score'].mean().sort_values(ascending=True)
    colors = ['red' if v < 60 else 'orange' if v < 80 else 'green' for v in subject_means.values]
    subject_means.plot(kind='barh', ax=ax2, color=colors)
    ax2.set_xlabel('평균 점수')
    ax2.set_ylabel('과목')
    ax2.set_title('과목별 평균 성적')
    ax2.axvline(x=60, color='red', linestyle='--', alpha=0.5)
    ax2.axvline(x=80, color='orange', linestyle='--', alpha=0.5)
    
    plt.tight_layout()
    plt.show()
else:
    print('과목별 데이터가 없습니다.')

## 4. 등급별 분포

In [ ]:
if not scores_df.empty and 'grade' in scores_df.columns:
    grade_counts = scores_df['grade'].value_counts().sort_index()
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    # 등급별 히스토그램
    colors = ['#2ecc71', '#27ae60', '#f1c40f', '#e67e22', '#e74c3c', 
              '#c0392b', '#9b59b6', '#8e44ad', '#34495e']
    grade_counts.plot(kind='bar', ax=ax1, color=[colors[i-1] for i in grade_counts.index])
    ax1.set_xlabel('등급')
    ax1.set_ylabel('횟수')
    ax1.set_title('등급별 분포')
    ax1.tick_params(axis='x', rotation=0)
    
    # 누적 분포
    cumulative = grade_counts.cumsum() / grade_counts.sum() * 100
    ax2.bar(range(1, 10), [cumulative.get(i, 0) for i in range(1, 10)], 
            color='steelblue', alpha=0.7)
    ax2.plot(range(1, 10), [cumulative.get(i, 0) for i in range(1, 10)], 
             'ro-', linewidth=2)
    ax2.set_xlabel('등급')
    ax2.set_ylabel('누적 비율 (%)')
    ax2.set_title('등급 누적 분포')
    ax2.set_xticks(range(1, 10))
    ax2.axhline(y=50, color='gray', linestyle='--', alpha=0.5)
    
    plt.tight_layout()
    plt.show()
    
    # 평균 등급 출력
    avg_grade = scores_df['grade'].mean()
    print(f'\n평균 등급: {avg_grade:.2f}')
else:
    print('등급 데이터가 없습니다.')

## 5. 취약 과목 분석

In [ ]:
if not scores_df.empty and 'subject' in scores_df.columns:
    # 과목별 통계
    subject_stats = scores_df.groupby('subject').agg({
        'score': ['mean', 'std', 'min', 'max', 'count']
    }).round(2)
    subject_stats.columns = ['평균', '표준편차', '최저', '최고', '횟수']
    subject_stats = subject_stats.sort_values('평균')
    
    print('과목별 성적 통계')
    print('=' * 60)
    display(subject_stats)
    
    # 취약 과목 (평균 60점 미만 또는 하위 3개)
    weak_subjects = subject_stats[subject_stats['평균'] < 60]
    if weak_subjects.empty:
        weak_subjects = subject_stats.head(3)
    
    print('\n취약 과목 (집중 학습 필요)')
    print('-' * 40)
    for subject, row in weak_subjects.iterrows():
        print(f'  - {subject}: 평균 {row["평균"]:.1f}점')
else:
    print('과목별 데이터가 없습니다.')

## 6. 학습-성적 상관관계

In [ ]:
# 학습 플랜 데이터 로드
plans_df = db.get_student_plans(STUDENT_ID)

if not plans_df.empty and not scores_df.empty:
    efficiency = calculate_study_efficiency(plans_df, scores_df)
    
    print('학습-성적 상관관계 분석')
    print('=' * 50)
    for key, value in efficiency.items():
        if isinstance(value, dict):
            print(f'\n{key.replace("_", " ").title()}:')
            for k, v in value.items():
                print(f'  - {k}: {v}')
        else:
            print(f'{key}: {value}')
else:
    print('상관관계 분석을 위한 데이터가 부족합니다.')

## 7. 종합 분석 리포트

In [ ]:
analysis = analyze_score_trends(scores_df)

print('=' * 60)
print('성적 트렌드 분석 종합 리포트')
print('=' * 60)

for key, value in analysis.items():
    print(f'\n### {key.replace("_", " ").title()}')
    if isinstance(value, dict):
        for k, v in value.items():
            if isinstance(v, dict):
                print(f'  {k}:')
                for kk, vv in v.items():
                    print(f'    - {kk}: {vv}')
            else:
                print(f'  - {k}: {v}')
    else:
        print(f'  {value}')

## 8. 개선 추천사항

In [ ]:
print('\n개선 추천사항')
print('=' * 50)

if 'weak_subjects' in analysis:
    print('\n1. 취약 과목 집중 학습')
    for subject, score in analysis['weak_subjects'].items():
        print(f'   - {subject}: 현재 평균 {score:.1f}점 → 목표 70점 이상')

if 'overall_trend' in analysis:
    trend = analysis['overall_trend']
    if trend['direction'] == 'declining':
        print('\n2. 성적 하락 추세 대응')
        print(f'   - 최근 {abs(trend["change"]):.1f}점 하락')
        print('   - 학습 방법 점검 필요')
    else:
        print('\n2. 성적 상승 추세 유지')
        print(f'   - 최근 {trend["change"]:.1f}점 상승')
        print('   - 현재 학습 패턴 유지 권장')

print('\n3. 학습 시간 최적화')
print('   - 집중력 높은 시간대에 취약 과목 배치')
print('   - 복습 주기 준수 (에빙하우스 망각곡선)')